# Google Cloud SQL for MySQL

> [Cloud SQL](https://cloud.google.com/sql) 是一项完全托管的关系数据库服务，具有高性能、无缝集成和出色的可扩展性。它提供 [MySQL](https://cloud.google.com/sql/mysql)、[PostgreSQL](https://cloud.google.com/sql/postgresql) 和 [SQL Server](https://cloud.google.com/sql/sqlserver) 数据库引擎。通过 Cloud SQL 的 Langchain 集成，扩展您的数据库应用程序以构建支持 AI 的体验。

本 Notebook 介绍如何使用 [Cloud SQL for MySQL](https://cloud.google.com/sql/mysql) 和 `MySQLLoader` 以及 `MySQLDocumentSaver` 来[保存、加载和删除 langchain 文档](/docs/how_to#document-loaders)。

在 [GitHub](https://github.com/googleapis/langchain-google-cloud-sql-mysql-python/) 上了解有关该软件包的更多信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-mysql-python/blob/main/docs/document_loader.ipynb)

## 开始之前

要运行此笔记本，您需要执行以下操作：

* [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
* [启用 Cloud SQL Admin API。](https://console.cloud.google.com/marketplace/product/google/sqladmin.googleapis.com)
* [创建 Cloud SQL for MySQL 实例](https://cloud.google.com/sql/docs/mysql/create-instance)
* [创建 Cloud SQL 数据库](https://cloud.google.com/sql/docs/mysql/create-manage-databases)
* [向数据库添加 IAM 数据库用户](https://cloud.google.com/sql/docs/mysql/add-manage-iam-users#creating-a-database-user) (可选)

在确认 Notebook 的运行时环境可以访问数据库后，请填充以下值并运行单元格，然后再运行示例脚本。

In [ ]:
# @markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance.
REGION = "us-central1"  # @param {type:"string"}
INSTANCE = "test-instance"  # @param {type:"string"}

# @markdown Please specify a database and a table for demo purpose.
DATABASE = "test"  # @param {type:"string"}
TABLE_NAME = "test-default"  # @param {type:"string"}

### 🦜🔗 库安装

集成功能位于其独立的 `langchain-google-cloud-sql-mysql` 包中，因此我们需要安装它。

In [ ]:
%pip install -upgrade --quiet langchain-google-cloud-sql-mysql

**仅限 Colab**: 取消注释下面的单元格以重启内核，或使用按钮重启内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### ☁ 设置您的 Google Cloud 项目
设置您的 Google Cloud 项目，以便在此 Notebook 中使用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 请参阅支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 🔐 身份验证

使用登录到此 notebook 的 IAM 用户身份验证 Google Cloud，以便访问您的 Google Cloud 项目。

- 如果您正在使用 Colab 运行此 notebook，请使用下面的单元格并继续。
- 如果您正在使用 Vertex AI Workbench，请查看此处的[设置说明](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [ ]:
from google.colab import auth

auth.authenticate_user()

## 基本用法

### MySQLEngine 连接池

在从 MySQL 表保存或加载文档之前，我们首先需要为 Cloud SQL 数据库配置一个连接池。`MySQLEngine` 可以为您的 Cloud SQL 数据库配置一个连接池，从而实现从您的应用程序成功连接并遵循行业最佳实践。

要使用 `MySQLEngine.from_instance()` 创建 `MySQLEngine`，您只需要提供 4 个信息：

1. `project_id`：Cloud SQL 实例所在 Google Cloud 项目的项目 ID。
2. `region`：Cloud SQL 实例所在的区域。
3. `instance`：Cloud SQL 实例的名称。
4. `database`：要连接到 Cloud SQL 实例上的数据库的名称。

默认情况下，将使用 [IAM 数据库身份验证](https://cloud.google.com/sql/docs/mysql/iam-authentication#iam-db-auth) 作为数据库身份验证的方法。此库使用属于[应用程序默认凭据 (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials) 的 IAM 主体，这些凭据源自环境。

有关 IAM 数据库身份验证的更多信息，请参阅：

* [为 IAM 数据库身份验证配置实例](https://cloud.google.com/sql/docs/mysql/create-edit-iam-instances)
* [使用 IAM 数据库身份验证管理用户](https://cloud.google.com/sql/docs/mysql/add-manage-iam-users)

此外，还可以使用[内置数据库身份验证](https://cloud.google.com/sql/docs/mysql/built-in-authentication)，通过用户名和密码访问 Cloud SQL 数据库。只需向 `MySQLEngine.from_instance()` 提供可选的 `user` 和 `password` 参数：

* `user`：用于内置数据库身份验证和登录的数据库用户名。
* `password`：用于内置数据库身份验证和登录的数据库密码。

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLEngine

engine = MySQLEngine.from_instance(
    project_id=PROJECT_ID, region=REGION, instance=INSTANCE, database=DATABASE
)

### 初始化表

通过 `MySQLEngine.init_document_table(<table_name>)` 初始化一个默认 schema 的表。表字段：

- page_content (类型: text)
- langchain_metadata (类型: JSON)

`overwrite_existing=True` 标志意味着新初始化的表将替换同名的现有表。

In [ ]:
engine.init_document_table(TABLE_NAME, overwrite_existing=True)

### 保存文档

使用 `MySQLDocumentSaver.add_documents(<documents>)` 保存 Langchain 文档。初始化 `MySQLDocumentSaver` 类需要提供两项内容：

1. `engine` - `MySQLEngine` 引擎的一个实例。
2. `table_name` - Cloud SQL 数据库中用于存储 Langchain 文档的表名。

In [ ]:
from langchain_core.documents import Document
from langchain_google_cloud_sql_mysql import MySQLDocumentSaver

test_docs = [
    Document(
        page_content="Apple Granny Smith 150 0.99 1",
        metadata={"fruit_id": 1},
    ),
    Document(
        page_content="Banana Cavendish 200 0.59 0",
        metadata={"fruit_id": 2},
    ),
    Document(
        page_content="Orange Navel 80 1.29 1",
        metadata={"fruit_id": 3},
    ),
]
saver = MySQLDocumentSaver(engine=engine, table_name=TABLE_NAME)
saver.add_documents(test_docs)

### 加载文档

使用 `MySQLLoader.load()` 或 `MySQLLoader.lazy_load()` 来加载 langchain 文档。`lazy_load` 返回一个生成器，该生成器仅在迭代期间查询数据库。要初始化 `MySQLLoader` 类，您需要提供：

1.  `engine` - `MySQLEngine` 引擎的一个实例。
2.  `table_name` - Cloud SQL 数据库中用于存储 langchain 文档的表名。

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLLoader

loader = MySQLLoader(engine=engine, table_name=TABLE_NAME)
docs = loader.lazy_load()
for doc in docs:
    print("Loaded documents:", doc)

### 通过查询加载文档

除了从表中加载文档，我们还可以选择从 SQL 查询生成的视图加载文档。例如：

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLLoader

loader = MySQLLoader(
    engine=engine,
    query=f"select * from `{TABLE_NAME}` where JSON_EXTRACT(langchain_metadata, '$.fruit_id') = 1;",
)
onedoc = loader.load()
onedoc

从 SQL 查询生成的视图可能具有与默认表不同的 schema。在这种情况下，MySQLLoader 的行为与从具有非默认 schema 的表加载相同。请参阅“”部分。

### 删除文档

使用 `MySQLDocumentSaver.delete(<documents>)` 从 MySQL 表中删除 Langchain 文档列表。

对于具有默认架构（page_content, langchain_metadata）的表，删除标准是：

如果列表中存在一个 `document`，并且满足以下条件，则应删除该 `row`：

- `document.page_content` 等于 `row[page_content]`
- `document.metadata` 等于 `row[langchain_metadata]`

In [ ]:
from langchain_google_cloud_sql_mysql import MySQLLoader

loader = MySQLLoader(engine=engine, table_name=TABLE_NAME)
docs = loader.load()
print("Documents before delete:", docs)
saver.delete(onedoc)
print("Documents after delete:", loader.load())

## 高级用法

### 加载文档，并自定义文档页面内容和元数据

首先，我们准备一个具有非默认架构的示例表，并用一些任意数据填充它。

In [ ]:
import sqlalchemy

with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{TABLE_NAME}`"))
    conn.commit()
    conn.execute(
        sqlalchemy.text(
            f"""
            CREATE TABLE IF NOT EXISTS `{TABLE_NAME}`(
                fruit_id INT AUTO_INCREMENT PRIMARY KEY,
                fruit_name VARCHAR(100) NOT NULL,
                variety VARCHAR(50),
                quantity_in_stock INT NOT NULL,
                price_per_unit DECIMAL(6,2) NOT NULL,
                organic TINYINT(1) NOT NULL
            )
            """
        )
    )
    conn.execute(
        sqlalchemy.text(
            f"""
            INSERT INTO `{TABLE_NAME}` (fruit_name, variety, quantity_in_stock, price_per_unit, organic)
            VALUES
                ('Apple', 'Granny Smith', 150, 0.99, 1),
                ('Banana', 'Cavendish', 200, 0.59, 0),
                ('Orange', 'Navel', 80, 1.29, 1);
            """
        )
    )
    conn.commit()

如果仍使用此示例表 `MySQLLoader` 的默认参数加载 langchain 文档，则加载的文档的 `page_content` 将是表的第一个列，而 `metadata` 将由所有其他列的键值对组成。

In [ ]:
loader = MySQLLoader(
    engine=engine,
    table_name=TABLE_NAME,
)
loader.load()

我们可以通过在初始化 `MySQLLoader` 时设置 `content_columns` 和 `metadata_columns` 来指定我们要加载的内容和元数据。

1.  `content_columns`: 要写入文档 `page_content` 的列。
2.  `metadata_columns`: 要写入文档 `metadata` 的列。

例如，在此示例中，`content_columns` 中列的值将连接成一个以空格分隔的字符串，作为加载文档的 `page_content`，而加载文档的 `metadata` 将只包含 `metadata_columns` 中指定列的键值对。

In [ ]:
loader = MySQLLoader(
    engine=engine,
    table_name=TABLE_NAME,
    content_columns=[
        "variety",
        "quantity_in_stock",
        "price_per_unit",
        "organic",
    ],
    metadata_columns=["fruit_id", "fruit_name"],
)
loader.load()

### 保存文档，包含自定义页面内容和元数据

为了将 langchain 文档以自定义元数据字段的形式保存到表中，我们首先需要通过 `MySQLEngine.init_document_table()` 创建这样一个表，并指定我们需要的 `metadata_columns` 列表。在此示例中，创建的表将包含以下列：

- description (类型: text): 用于存储水果描述。
- fruit_name (类型: text): 用于存储水果名称。
- organic (类型: tinyint(1)): 用于指示水果是否为有机。
- other_metadata (类型: JSON): 用于存储水果的其他元数据信息。

我们可以使用 `MySQLEngine.init_document_table()` 的以下参数来创建表：

1. `table_name`: 用于在 Cloud SQL 数据库中存储 langchain 文档的表名。
2. `metadata_columns`: 一个 `sqlalchemy.Column` 列表，指示所需的元数据列。
3. `content_column`: 用于存储 langchain 文档的 `page_content` 的列名。默认为 `page_content`。
4. `metadata_json_column`: 用于存储 langchain 文档额外 `metadata` 的 JSON 列名。默认为 `langchain_metadata`。

In [ ]:
engine.init_document_table(
    TABLE_NAME,
    metadata_columns=[
        sqlalchemy.Column(
            "fruit_name",
            sqlalchemy.UnicodeText,
            primary_key=False,
            nullable=True,
        ),
        sqlalchemy.Column(
            "organic",
            sqlalchemy.Boolean,
            primary_key=False,
            nullable=True,
        ),
    ],
    content_column="description",
    metadata_json_column="other_metadata",
    overwrite_existing=True,
)

使用 `MySQLDocumentSaver.add_documents(<documents>)` 保存文档。正如您在此示例中看到的，

- `document.page_content` 将被保存到 `description` 列。
- `document.metadata.fruit_name` 将被保存到 `fruit_name` 列。
- `document.metadata.organic` 将被保存到 `organic` 列。
- `document.metadata.fruit_id` 将以 JSON 格式保存到 `other_metadata` 列。

In [ ]:
test_docs = [
    Document(
        page_content="Granny Smith 150 0.99",
        metadata={"fruit_id": 1, "fruit_name": "Apple", "organic": 1},
    ),
]
saver = MySQLDocumentSaver(
    engine=engine,
    table_name=TABLE_NAME,
    content_column="description",
    metadata_json_column="other_metadata",
)
saver.add_documents(test_docs)

In [ ]:
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text(f"select * from `{TABLE_NAME}`;"))
    print(result.keys())
    print(result.fetchall())

### 删除具有自定义页面内容和元数据的文档

我们还可以通过 `MySQLDocumentSaver.delete(<documents>)` 从具有自定义元数据列的表中删除文档。删除条件为：

如果列表中存在一个 `document`，满足以下条件，则应删除一行 `row`：

- `document.page_content` 等于 `row[page_content]`
- 对于 `document.metadata` 中的每个元数据字段 `k`
    - `document.metadata[k]` 等于 `row[k]` 或 `document.metadata[k]` 等于 `row[langchain_metadata][k]`
- `row` 中不存在但 `document.metadata` 中不存在的额外元数据字段。

In [ ]:
loader = MySQLLoader(engine=engine, table_name=TABLE_NAME)
docs = loader.load()
print("Documents before delete:", docs)
saver.delete(docs)
print("Documents after delete:", loader.load())